In [ ]:
import os
import numpy as np

import keras
from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten, Conv2D, MaxPooling2D
from sklearn.metrics import accuracy_score
from tensorflow.keras.utils import to_categorical

## Download dataset 
https://drive.google.com/drive/u/3/folders/1sHh6NvuKX6RB5OytLwf4kaqfQ9svJNDQ

## Load data

Cause I use Colab to train my model, I need to add following codes to access the data. 

In [ ]:
# Install a Drive FUSE wrapper.
# https://github.com/astrada/google-drive-ocamlfuse
!apt-get install - y - qq software-properties-common python-software-properties module-init-tools
!add-apt-repository - y ppa: alessandro-strada/ppa 2 > &1 > /dev/null
!apt-get update - qq 2 > &1 > /dev/null
!apt-get - y install - qq google-drive-ocamlfuse fuse

E: Package 'python-software-properties' has no installation candidate
Selecting previously unselected package google-drive-ocamlfuse.
(Reading database ... 160772 files and directories currently installed.)
Preparing to unpack .../google-drive-ocamlfuse_0.7.26-0ubuntu1~ubuntu18.04.1_amd64.deb ...
Unpacking google-drive-ocamlfuse (0.7.26-0ubuntu1~ubuntu18.04.1) ...
Setting up google-drive-ocamlfuse (0.7.26-0ubuntu1~ubuntu18.04.1) ...
Processing triggers for man-db (2.8.3-2ubuntu0.1) ...


In [ ]:
# Generate auth tokens for Colab

from google.colab import auth
auth.authenticate_user()

In [ ]:
# Generate creds for the Drive FUSE library.

import getpass
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
!google-drive-ocamlfuse - headless - id = {creds.client_id} - secret = {creds.client_secret} < /dev/null 2 > &1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse - headless - id = {creds.client_id} - secret = {creds.client_secret}

Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
··········
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
Please enter the verification code: Access token retrieved correctly.


In [ ]:
# Create a directory and mount Google Drive using that directory.

!mkdir - p drive
!google-drive-ocamlfuse drive

In [ ]:
print('Files in Drive:')
!ls drive/

Files in Drive:
'Colab Notebooks'


In [ ]:
x_train = np.load("drive/Colab Notebooks/x_train.npy")
y_train = np.load("drive/Colab Notebooks/y_train.npy")

x_test = np.load("drive/Colab Notebooks/x_test.npy")
y_test = np.load("drive/Colab Notebooks/y_test.npy")

print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

50000 train samples
10000 test samples


In [ ]:
# It's a multi-class classification problem
class_index = {'airplane': 0, 'automobile': 1, 'bird': 2, 'cat': 3, 'deer': 4,
               'dog': 5, 'frog': 6, 'horse': 7, 'ship': 8, 'truck': 9}
print(np.unique(y_train))

[0 1 2 3 4 5 6 7 8 9]


![image](https://img-blog.csdnimg.cn/20190623084800880.png?x-oss-process=image/watermark,type_ZmFuZ3poZW5naGVpdGk,shadow_10,text_aHR0cHM6Ly9ibG9nLmNzZG4ubmV0L3lqcDE5ODcxMDEz,size_16,color_FFFFFF,t_70)

## Data preprocess

In [ ]:
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
#x_train /= 255
#x_test /= 255

mean = np.mean(x_train, axis=(0, 1, 2, 3))
std = np.std(x_train, axis=(0, 1, 2, 3))
x_train = (x_train-mean)/(std+1e-7)
x_test = (x_test-mean)/(std+1e-7)

# Convert class vectors to one-hot encoding (keras model requires one-hot label as inputs)
num_classes = 10
print(y_train[0])
y_train = to_categorical(y_train, num_classes)
y_test = to_categorical(y_test, num_classes)
print(y_train[0])

[9]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]


# ImageDataGenerator

In [ ]:
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from keras.callbacks import ReduceLROnPlateau

train_datagen = ImageDataGenerator(rotation_range=40,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True,
                                   fill_mode="nearest")


train_datagen.fit(x_train)

# Set the learing rate reducing schedule

In [ ]:
def lr_schedule(epoch):
    lrate = 0.001
    if epoch > 75:
        lrate = 0.0005
    if epoch > 100:
        lrate = 0.0003
    return lrate

# Using  Fractional Max-Pooling

In [ ]:
def frac_max_pool(x):
    return tf.nn.fractional_max_pool(x, [1.0, 1.41, 1.41, 1.0], pseudo_random=True, overlapping=True)[0]

## Build model & training (Keras)

In [ ]:
# Builde model
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from keras.layers import Dropout
from keras.layers import BatchNormalization
from keras.layers import Input, Lambda
from keras import regularizers, optimizers
from keras.callbacks import LearningRateScheduler

baseMapNum = 32
weight_decay = 1e-4

model = Sequential()  # Sequential groups a linear stack of layers
model.add(Conv2D(baseMapNum, (3, 3), padding='same', kernel_regularizer=regularizers.l2(
    weight_decay), input_shape=x_train.shape[1:]))
model.add(Activation('relu'))  # Add Relu activation for non-linearity
model.add(BatchNormalization())
model.add(Conv2D(baseMapNum, (3, 3), padding='same',
          kernel_regularizer=regularizers.l2(weight_decay)))
model.add(Activation('relu'))  # Add Relu activation for non-linearity
model.add(BatchNormalization())
model.add(Lambda(frac_max_pool))
model.add(Dropout(0.1))


model.add(Conv2D(2*baseMapNum, (3, 3), padding='same',
          kernel_regularizer=regularizers.l2(weight_decay)))
model.add(Activation('relu'))  # Add Relu activation for non-linearity
model.add(BatchNormalization())
model.add(Conv2D(2*baseMapNum, (3, 3), padding='same',
          kernel_regularizer=regularizers.l2(weight_decay)))
model.add(Activation('relu'))  # Add Relu activation for non-linearity
model.add(BatchNormalization())
model.add(Lambda(frac_max_pool))
model.add(Dropout(0.2))


model.add(Conv2D(4*baseMapNum, (3, 3), padding='same',
          kernel_regularizer=regularizers.l2(weight_decay)))
model.add(Activation('relu'))  # Add Relu activation for non-linearity
model.add(BatchNormalization())
model.add(Conv2D(4*baseMapNum, (3, 3), padding='same',
          kernel_regularizer=regularizers.l2(weight_decay)))
model.add(Activation('relu'))  # Add Relu activation for non-linearity
model.add(BatchNormalization())
model.add(Lambda(frac_max_pool))
model.add(Dropout(0.3))


model.add(Conv2D(8*baseMapNum, (3, 3), padding='same',
          kernel_regularizer=regularizers.l2(weight_decay)))
model.add(Activation('relu'))  # Add Relu activation for non-linearity
model.add(BatchNormalization())
model.add(Conv2D(8*baseMapNum, (3, 3), padding='same',
          kernel_regularizer=regularizers.l2(weight_decay)))
model.add(Activation('relu'))  # Add Relu activation for non-linearity
model.add(BatchNormalization())
model.add(Lambda(frac_max_pool))
model.add(Dropout(0.4))


model.add(Flatten())  # Flatten the featuremaps
model.add(Dense(units=512))  # Add dense layer with 512 neurons
model.add(Activation('relu'))  # Add Relu activation for non-linearity
model.add(BatchNormalization())
model.add(Dense(units=512))  # Add dense layer with 512 neurons
model.add(Activation('relu'))  # Add Relu activation for non-linearity
model.add(BatchNormalization())
model.add(Dropout(0.5))
model.add(Dense(units=num_classes))  # Add final output layer for 10 classes
# Add softmax activation to transfer logits into probabilities
model.add(Activation('softmax'))

# initiate Nadam optimizer
opt = keras.optimizers.Nadam(
    learning_rate=0.001, beta_1=0.9, beta_2=0.999, epsilon=None, schedule_decay=1e-6)
# I tried different  optimizers
#opt = keras.optimizers.Adam(learning_rate=0.0008, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.004, amsgrad=False)
#opt = keras.optimizers.SGD(learning_rate=0.001, momentum=0.9)
#opt = keras.optimizers.RMSprop(learning_rate=0.0001, decay=1e-6)
#opt = keras.optimizers.RMSprop(learning_rate=0.001,decay=1e-6)


# Compile the model with loss function and optimizer, and evaluate with accuracy
model.compile(loss='categorical_crossentropy',
              optimizer=opt, metrics=['accuracy'])

history = model.fit(train_datagen.flow(x_train, y_train, batch_size=64),
                    steps_per_epoch=(len(x_train))/64,
                    epochs=125,
                    verbose=1,
                    validation_data=(x_test, y_test),
                    validation_steps=(len(x_test))/64,
                    callbacks=[LearningRateScheduler(lr_schedule)],
                    shuffle=True)

score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Epoch 1/125
781/781 [==============================] - 116s 142ms/step - loss: 2.3813 - accuracy: 0.2880 - val_loss: 1.7905 - val_accuracy: 0.4476
Epoch 2/125
781/781 [==============================] - 109s 140ms/step - loss: 1.6002 - accuracy: 0.4589 - val_loss: 1.5158 - val_accuracy: 0.5499
Epoch 3/125
781/781 [==============================] - 110s 140ms/step - loss: 1.3824 - accuracy: 0.5349 - val_loss: 1.3039 - val_accuracy: 0.5815
Epoch 4/125
781/781 [==============================] - 109s 139ms/step - loss: 1.2853 - accuracy: 0.5813 - val_loss: 1.0721 - val_accuracy: 0.6588
Epoch 5/125
781/781 [==============================] - 108s 138ms/step - loss: 1.1956 - accuracy: 0.6151 - val_loss: 1.2043 - val_accuracy: 0.6409
Epoch 6/125
781/781 [==============================] - 107s 137ms/step - loss: 1.1407 - accuracy: 0.6411 - val_loss: 1.0011 - val_accuracy: 0.7043
Epoch 7/125
781/781 [==============================] - 105s 135ms/step - loss: 1.1055 - accuracy: 0.6639 - val_loss: 1

In [ ]:
y_pred = model.predict(x_test)
print(y_pred.shape)  # 10000 samples, each sample with probaility of 10 classes

(10000, 10)


In [ ]:
y_pred[0]

array([9.0096257e-13, 9.9697077e-01, 1.2179741e-10, 3.5244339e-09,
       6.8526731e-09, 6.0757920e-08, 2.2757572e-10, 1.1206939e-07,
       8.8837097e-13, 3.0289458e-03], dtype=float32)

In [ ]:
# argmax to find the predict class with highest probability. 9=truck
np.argmax(y_pred[0])

1

## DO NOT MODIFY CODE BELOW!
**Please screen shot your results and post it on your report**

In [ ]:
# Change the label to the number of class
y_pred = model.predict(x_test)
y_pred = np.argmax(y_pred, axis=1)

In [ ]:
assert y_pred.shape == (10000,)

In [ ]:
y_test = np.load("drive/Colab Notebooks/y_test.npy")
print("Accuracy of my model on test set: ", accuracy_score(y_test, y_pred))

Accuracy of my model on test set:  0.9012
